In [1]:
import os
import plistlib
from base64 import b64encode, b64decode
import pandas
from datetime import datetime
import pgpy
import html
import json
from hexdump import hexdump
import ccl_bplist
from io import BytesIO
from Crypto.Cipher import AES
import sqlite3
import hashlib

In [2]:
#Constants

out_path = '../mnt2/tests_proton/'
extraction_path = '../mnt2/tests_proton/PIN'
db_name = 'ProtonMail.sqlite'
plist_name = 'group.ch.protonmail.protonmail.plist'
keychain_plist_path = os.path.join(out_path,'PIN','keychain_decrypted.plist')
IVsize = 16
pin=b'123456'

#making a list of files in path
files=[]
for r, d, f in os.walk(extraction_path):
    for file in f:
         files.append(os.path.join(r, file))


### Extracting Private Key
#### Getting Protonmail values from decrypted keychain

In [3]:
with open(keychain_plist_path,'rb') as f :
    plist = plistlib.load(f)
    
keychainVal={}
if type(plist) == list:
    for dd in plist:
        if type(dd) == dict:        
            if 'svce' in dd:
                if 'protonmail' in str(dd['svce']) :
                    #print(dd)
                    keychainVal[dd['acct']]=dd['v_Data']
else:
    for d in plist:
        for dd in plist[d]:
            if type(dd) == dict:        
                if 'svce' in dd:
                    if 'protonmail' in str(dd['svce']):
                        #print(dd)
                        keychainVal[dd['acct']]=dd['v_Data']


#### Decrypt and extract private Key

In [7]:
"""
keys (except Master) are in bplist in Preference along with DB
decrypt plist with MainKey, parse bplist and extract privateKeyCoderKey 
"""

def getMainKeyFromPinProtection(pin):
    salt = keychainVal['PinProtection.salt']
    encMainKey = keychainVal['PinProtection']
    
    ikey = hashlib.scrypt(pin, salt=salt, n=32768, r=8, p=1, maxmem=33*1024*1024, dklen=32)
    cipher = AES.new(ikey, AES.MODE_CTR, initial_value=encMainKey[:IVsize], nonce=b'')
    mainKey = bytes(plistlib.loads(cipher.decrypt(encMainKey[IVsize:])))

    
if 'NoneProtection' in keychainVal:
    mainKey = keychainVal['NoneProtection']
elif 'PinProtection' in keychainVal:
    if len(pin) == 0:
        print('Need to crack pin first')
        
    mainKey = getMainKeyFromPinProtection(pin)
    

def decryptWithMainKey(encrypted):
    iv = encrypted[:IVsize]
    cipher = AES.new(mainKey, AES.MODE_CTR, initial_value=iv, nonce=b'')
    return cipher.decrypt(encrypted[IVsize:])

for fi in [f for f in files if f.endswith(plist_name)]:
    with open(fi,'rb') as p :
        prefplist = plistlib.load(p)

if mainKey:
    enc_val = prefplist['authKeychainStoreKeyProtectedWithMainKey']
    dec_val = decryptWithMainKey(enc_val)

    keychainStorePlist1 = ccl_bplist.load(BytesIO(dec_val))
    keychainStorePlist = ccl_bplist.load(BytesIO(keychainStorePlist1[0]))
    keychainStore = ccl_bplist.deserialise_NsKeyedArchiver(keychainStorePlist, parse_whole_structure=True)
    privateKeyCoderKey = keychainStore['root']['NS.objects'][0]['privateKeyCoderKey']

### Getting data from DB

In [92]:
try:
    db_file = [f for f in files if f.endswith(db_name)][0]
except:
    print('File {} not found in provided path'.format(db_name))
    
db = sqlite3.connect(db_file)

df = pandas.read_sql_query("""
SELECT
    m.ZTIME,
    m.ZBODY,
    m.ZMIMETYPE,
    m.ZTOLIST,
    m.ZREPLYTOS,
    m.ZSENDER,
    m.ZTITLE,
    m.ZISENCRYPTED
FROM
    ZMESSAGE m
""",db)

df['datetime'] = pandas.to_datetime(df['ZTIME'], origin=pandas.Timestamp('2001-01-01'), unit='s')

### Decrypting message

In [136]:
#load private key

key, _ = pgpy.PGPKey.from_blob(privateKeyCoderKey)

pwdKey = keychainStore['root']['NS.objects'][0]['AuthCredential.Password']

def decrypt_message(encm):
    if('-----BEGIN PGP MESSAGE-----') in encm:
        with key.unlock(pwdKey):
            assert key.is_unlocked
            message_from_blob = pgpy.PGPMessage.from_blob(encm)
            decm = key.decrypt(message_from_blob).message
            #print(decm)
            return html.unescape(decm.encode('cp1252', errors='ignore').decode('utf8', errors='ignore'))
    else:
        return encm
    
df['decrypted_message'] = df['ZBODY'].apply(decrypt_message)


def parse_sender_recipient(row):
    sender = json.loads(plistlib.loads(decryptWithMainKey(row['ZSENDER']))[0])
    to = json.loads(plistlib.loads(decryptWithMainKey(row['ZTOLIST']))[0])
    row['Sender'] = "{} <{}>".format(sender['Name'],sender['Address'])
    row['Recipients'] = ""
    for r in to:
        row['Recipients'] += "{} <{}><br />".format(r['Name'],r['Address'])
    row['Title'] = plistlib.loads(decryptWithMainKey(df.iloc[0]['ZTITLE']))[0]
    return row

df = df.apply(lambda r:parse_sender_recipient(r), axis=1)

## Export as dynamic table

In [ ]:
from dynamic_table import *

In [ ]:
df_export = df[['datetime','Sender','Recipients','Title','decrypted_message']]

df_export = df_export.rename(columns={
    "datetime":"Date/Heure (UTC)",
    "Sender":"Expéditeur",
    "Recipients":"Destinataires",
    "Title":"Sujet",
    "decrypted_message":"Message"
})

html_str = html_head +table_header_part1
table_cols = ''
for c in df_export.columns.values:
    table_cols += table_header_col.format(c,c)
    
html_str += table_cols + table_header_part2

html_rows=''
for index,row in df_export.iterrows():
    html_rows += table_body_row.format(index)
    for _,v in row.items():
        html_rows += '<td>{}</td>'.format(v)
    html_rows += '</tr>'

html_str += html_rows + table_body_footer + js

with open(os.path.join(out_path,'protonmail_table.html'),'w') as out:
    out.write(html)
    out.write('</body></html>')

In [ ]:
df_export.to_excel(os.path.join(out_path,'messages_protonmail.xlsx'), sheet_name="ProtonMail", index=False)

## Bruteforce PIN

In [17]:
import multiprocessing
import itertools
from tqdm.contrib.concurrent import process_map

charset="1234567890"

salt = keychainVal['PinProtection.salt']
encMainKey = keychainVal['PinProtection']
    
# Number of digits of the PIN
numCar = 6
    
def worker(pin):
    ikey = hashlib.scrypt(pin, salt=salt, n=32768, r=8, p=1, maxmem=33*1024*1024, dklen=32)
    cipher = AES.new(ikey, AES.MODE_CTR, initial_value=encMainKey[:IVsize], nonce=b'')
    mainKey = cipher.decrypt(encMainKey[IVsize:])

    if b'bplist' in mainKey:
        print('found :%s'%(pin))
        #break
    return

def iterator(numCar):
    start = 0
    for n in range(0,10**numCar):
        formatstring = '{'+':0{}d'.format(numCar)+'}'
        yield(bytes(formatstring.format(n).encode('utf8')))
    
with multiprocessing.Pool(10) as workers:
    total = 10**numCar
    candidates = itertools.product(charset, repeat=6)
    r = process_map(worker, 
                    iterator(numCar), 
                    max_workers=10, 
                    chunksize=100, 
                    total=total
                   )

    

found :b'123456'

